In [2]:
import numpy as np
import cv2
import os
from matplotlib import pyplot as plt
# Load the Haar Cascade XML file for face detection
face_cascade = cv2.CascadeClassifier(r'haar_xml\raw.githubusercontent.com_opencv_opencv_master_data_haarcascades_haarcascade_frontalface_default.xml')
eyes_cascade = cv2.CascadeClassifier(r'haar_xml\haarcascade_eye.xml')

def get_intraocular_distance_in_pixels(lefteye, righteye):
    leftx, lefty = lefteye[0]+lefteye[2]/2, lefteye[1]+lefteye[3]/2
    point1 = np.array((leftx,lefty))
    rightx, righty = righteye[0]+righteye[2]/2, righteye[1]+righteye[3]/2
    point2 = np.array((rightx,righty))
    dist = np.sqrt(np.sum(np.square(point1-point2)))
    return dist

In [3]:
def resize_keep_aspect_ratio(image, length_of_largest_dim=300, interpolation=cv2.INTER_AREA):
    if image.shape[1]>=image.shape[0]:
        ratio = image.shape[0]/image.shape[1]
        dim = (length_of_largest_dim, int(length_of_largest_dim*ratio))
    else:
        ratio = image.shape[1]/image.shape[0]
        dim = (int(length_of_largest_dim*ratio), length_of_largest_dim)
    return cv2.resize(image, dim, interpolation = interpolation)

In [4]:
avg_eye_width = 35
templates = [cv2.imread(rf'iris_templates\{img_file}') for img_file in os.listdir('iris_templates')]

In [5]:
def sem_preprocessamento(img):
    return img

In [27]:
def preprocessamento1(img):
    img = resize_keep_aspect_ratio(img, length_of_largest_dim=300) #resize mantendo ratio entre lados
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #converte pra grayscale
    img = cv2.GaussianBlur(img,(3,3), sigmaX=33, sigmaY=33) #aplica blur gaussiano
    return img
# preprocessamento1(cv2.imread(r'data\Natalie Portman\069_c9e3207d.jpg'))

In [28]:
def preprocessamento2(img):
    # img = resize_keep_aspect_ratio(img, length_of_largest_dim=512) #tamanho original
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #converte pra grayscale
    # img = cv2.GaussianBlur(img,(3,3), sigmaX=33, sigmaY=33) #sem blur gaussiano
    return img

# for person in os.listdir('data'):
#     for img_file in os.listdir(f'data\{person}'):
#         image = cv2.imread(rf'data\{person}\{img_file}')
#         image = preprocessamento1(image)
#         cv2.imshow('Image with Detected Faces', image)
#         cv2.waitKey(0)
#         cv2.destroyAllWindows()

In [44]:
def preprocessamento3(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #converte pra grayscale
    img = cv2.bilateralFilter(img,5,50,50) #blur bilateral
    return img
preprocessamento3(cv2.imread(r'data\Natalie Portman\069_c9e3207d.jpg'))


array([[241, 241, 239, ..., 230, 232, 233],
       [241, 240, 238, ..., 230, 232, 233],
       [239, 239, 236, ..., 228, 230, 231],
       ...,
       [ 30,  26,  21, ...,  27,  28,  29],
       [ 34,  25,  22, ...,  30,  32,  40],
       [ 33,  26,  22, ...,  30,  35,  40]], dtype=uint8)

In [ ]:
methods = ['cv2.TM_SQDIFF']
eye_over_face_ratio = 50/315
for person in os.listdir('data'):
    for img_file in os.listdir(f'data\{person}'):
        image = cv2.imread(rf'data\{person}\{img_file}')
        image = preprocessamento2(image)
        faces = face_cascade.detectMultiScale(image, scaleFactor=1.1, minNeighbors=5)
        for (x, y, w, h) in faces:
            # cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            roi = image[y:y+h, x:x+w]
            # resized_face = cv2.resize(roi, (150,150), interpolation = cv2.INTER_AREA)
            # cv2.imshow('Image with Detected Faces', image)
            # cv2.waitKey(0)
            # cv2.destroyAllWindows()
            # roi_gray = gray_image[y:y+h, x:x+w]
            tolerance = .1
        eyes = eyes_cascade.detectMultiScale(roi, minNeighbors=2, minSize=(int(roi.shape[0]*(eye_over_face_ratio*(1-tolerance))),int(roi.shape[1]*(eye_over_face_ratio*(1-tolerance)))), maxSize=(int(roi.shape[0]*(eye_over_face_ratio*(1+tolerance))),int(roi.shape[1]*(eye_over_face_ratio*(1+tolerance)))))

        for (ex,ey,ew,eh) in eyes:
            roi_eye = roi[ey:ey+eh, ex:ex+ew]
            eye_over_template_ratio = ew/templates[0].shape[0]
            iris_over_eye_ratio = 0.31
            # cv2.rectangle(image,(ex+x,ey+y),(ex+ew+x,ey+eh+y),(0,255,0),2)
            
            # cv2.imshow('Image with Detected Faces', image)
            # cv2.waitKey(0)
            # cv2.destroyAllWindows()
            # cv2.waitKey(0)
            # cv2.destroyAllWindows()
            for idx,template in enumerate(templates):
                template = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY) #converte template pra grayscale
                new_template_shape = (int(template.shape[0]*eye_over_template_ratio*iris_over_eye_ratio),int(template.shape[1]*eye_over_template_ratio*iris_over_eye_ratio))
                resized_template = cv2.resize(template, new_template_shape, interpolation = cv2.INTER_AREA)
                tw, th = resized_template.shape[::-1]
                
                for meth in methods:
                    method = eval(meth)
                    # Apply template Matching
                    res = cv2.matchTemplate(roi_eye,resized_template,method)
                    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)

                    # If the method is TM_SQDIFF or TM_SQDIFF_NORMED, take minimum
                    if method in [cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED]:
                        top_left = min_loc
                    else:
                        top_left = max_loc
                    bottom_right = (top_left[0] + tw, top_left[1] + th)
                    roi_rec = roi_eye.copy()
                    cv2.rectangle(roi_rec,top_left, bottom_right, 255, 2)

                    plt.subplot(121),plt.imshow(res,cmap = 'gray')
                    plt.title('Matching Result'), plt.xticks([]), plt.yticks([])
                    plt.subplot(122),plt.imshow(roi_rec,cmap = 'gray')
                    plt.title('Detected Point'), plt.xticks([]), plt.yticks([])
                    plt.suptitle(f'Template {idx+1}')
                    plt.show()

                # x_offset=y_offset=0
                # roi_eye[y_offset:y_offset+resized_template.shape[0], x_offset:x_offset+resized_template.shape[1]] = resized_template
                
            # cv2.imshow('Image with Detected Faces', res)
            # cv2.waitKey(0)
            # cv2.destroyAllWindows()
            # cv2.imshow('Image with Detected Faces', roi_eye)
            # cv2.waitKey(0)
            # cv2.destroyAllWindows()
            # loc = np.where(res >= threshold)
            # tw, th = template.shape[::-1]
            # for pt in zip(*loc[::-1]):
                # cv2.rectangle(roi_eye, pt, (pt[0] + tw, pt[1] + th), (255, 255, 255), 2)
            
            #cv2.rectangle(image,(ex+x,ey+y),(ex+ew+x,ey+eh+y),(0,255,0),2)
                # cv2.imshow('Image with Detected Faces', res)
                # cv2.waitKey(0)
                # cv2.destroyAllWindows()
        # cv2.imshow('Image with Detected Faces', image)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()

# [44,44,30,32,29,25,35,31,36,38,37,37,29,28,34,36,42,41,41,43,43,44,41,42,35,31,34,30,24,31,33,31,26,24,36,30,35,36,38,41]


In [19]:
resultados_pipe_1 = [0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,1,1,1,0,1,0,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,1,0,1,0,0,0,0,0,1,0,1,1,0,0,1,1]
resultados_pipe_1_template_1 = resultados_pipe_1[::4]
resultados_pipe_1_template_2 = resultados_pipe_1[1::4]
resultados_pipe_1_template_3 = resultados_pipe_1[2::4]
resultados_pipe_1_template_4 = resultados_pipe_1[3::4]
pipe1_temp1_acc = np.count_nonzero(resultados_pipe_1_template_1)/len(resultados_pipe_1_template_1)
pipe1_temp2_acc = np.count_nonzero(resultados_pipe_1_template_2)/len(resultados_pipe_1_template_2)
pipe1_temp3_acc = np.count_nonzero(resultados_pipe_1_template_3)/len(resultados_pipe_1_template_3)
pipe1_temp4_acc = np.count_nonzero(resultados_pipe_1_template_4)/len(resultados_pipe_1_template_4)
print('pipe1temp1',pipe1_temp1_acc)
print('pipe1temp2',pipe1_temp2_acc)
print('pipe1temp3',pipe1_temp3_acc)
print('pipe1temp4',pipe1_temp4_acc)

resultados_sem_preproc = [0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,]
resultados_sem_preproc_template_1 = resultados_sem_preproc[::4]
resultados_sem_preproc_template_2 = resultados_sem_preproc[1::4]
resultados_sem_preproc_template_3 = resultados_sem_preproc[2::4]
resultados_sem_preproc_template_4 = resultados_sem_preproc[3::4]
sem_preproc_temp1_acc = np.count_nonzero(resultados_sem_preproc_template_1)/len(resultados_sem_preproc_template_1)
sem_preproc_temp2_acc = np.count_nonzero(resultados_sem_preproc_template_2)/len(resultados_sem_preproc_template_2)
sem_preproc_temp3_acc = np.count_nonzero(resultados_sem_preproc_template_3)/len(resultados_sem_preproc_template_3)
sem_preproc_temp4_acc = np.count_nonzero(resultados_sem_preproc_template_4)/len(resultados_sem_preproc_template_4)
print('sem preproc temp1', sem_preproc_temp1_acc)
print('sem preproc temp2', sem_preproc_temp2_acc)
print('sem preproc temp3', sem_preproc_temp3_acc)
print('sem preproc temp4', sem_preproc_temp4_acc)

pipe1temp1 0.5813953488372093
pipe1temp2 0.0
pipe1temp3 0.5581395348837209
pipe1temp4 0.11627906976744186
sem preproc temp1 0.6666666666666666
sem preproc temp2 0.0
sem preproc temp3 0.5
sem preproc temp4 0.06666666666666667


In [43]:
resultados_pipe_3 = [0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,0,0,1,0,1,0,1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,0,0,1,0,1,0,1,0,1,0,1,0,0,0,1,1,0,0,0,0]
resultados_pipe_3_template_1 = resultados_pipe_3[::4]
resultados_pipe_3_template_2 = resultados_pipe_3[1::4]
resultados_pipe_3_template_3 = resultados_pipe_3[2::4]
resultados_pipe_3_template_4 = resultados_pipe_3[3::4]
pipe_3_temp1_acc = np.count_nonzero(resultados_pipe_3_template_1)/len(resultados_pipe_3_template_1)
pipe_3_temp2_acc = np.count_nonzero(resultados_pipe_3_template_2)/len(resultados_pipe_3_template_2)
pipe_3_temp3_acc = np.count_nonzero(resultados_pipe_3_template_3)/len(resultados_pipe_3_template_3)
pipe_3_temp4_acc = np.count_nonzero(resultados_pipe_3_template_4)/len(resultados_pipe_3_template_4)
print('sem preproc temp1', pipe_3_temp1_acc)
print('sem preproc temp2', pipe_3_temp2_acc)
print('sem preproc temp3', pipe_3_temp3_acc)
print('sem preproc temp4', pipe_3_temp4_acc)

sem preproc temp1 0.6388888888888888
sem preproc temp2 0.0
sem preproc temp3 0.5277777777777778
sem preproc temp4 0.05555555555555555


In [4]:


for person in os.listdir('data'):
    for img in os.listdir(f'data\{person}'):
        
        image_path = rf'data\{person}\{img}'
        image = cv2.imread(image_path)      
          
        if image is None:
            print(f"Error: Unable to load image {image_path}.")
            
        else:
        # Resize and convert the image to grayscale for face detection
            image = resize_keep_aspect_ratio(image, length_of_largest_dim=400)
        
            gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            
            # Perform face detection
            faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

            # Draw rectangles around the detected faces
            for (x, y, w, h) in faces:
                cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
                roi_color = image[y:y+h, x:x+w]
                roi_gray = gray_image[y:y+h, x:x+w]

            eyes = eyes_cascade.detectMultiScale(roi_gray, minNeighbors=2)
            for (ex,ey,ew,eh) in eyes:
                cv2.rectangle(image,(ex+x,ey+y),(ex+ew+x,ey+eh+y),(0,255,0),2)

            # Display the image with detected faces
            cv2.imshow('Image with Detected Faces', image)
            cv2.waitKey(0)
            cv2.destroyAllWindows()

In [331]:
eyes_list = []
faces_list = []
names_list = []
for person in os.listdir(r'archive\Celebrity Faces Dataset'):
    for img in os.listdir(rf'archive\Celebrity Faces Dataset\{person}'):
        image_path = rf'archive\Celebrity Faces Dataset\{person}\{img}'
        image = cv2.imread(image_path)      
        if image is None:
            raise Exception('Erro ao carregar imagem')
        else:
            names_list.append(person)
            image = resize_keep_aspect_ratio(image, length_of_largest_dim=256)
            gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            
            faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
            for (x, y, w, h) in faces:
                # cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
                roi_color = image[y:y+h, x:x+w]
                roi_gray = gray_image[y:y+h, x:x+w]
            
            roi_color = cv2.resize(roi_color, (128,128), interpolation=cv2.INTER_AREA)
            faces_list.append(roi_color)
            roi_gray = cv2.resize(roi_gray, (128,128), interpolation=cv2.INTER_AREA)
            eyes = eyes_cascade.detectMultiScale(roi_gray)#, minNeighbors=2, minSize=(8,8), maxSize=(25,25), )
            eyes_list.append(eyes)
            # for (ex,ey,ew,eh) in eyes:
                # cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
                # cv2.rectangle(image,(ex+x,ey+y),(ex+ew+x,ey+eh+y),(0,255,0),2)
                    

In [332]:
def get_eye_center(eye):
    x,y,w,h = eye
    eye_center = (x + w//2, y + h//2)
    return eye_center


In [340]:
#caso tenha sido detectado mais de dois olhos numa mesma imagem, seleciona os dois mais provaveis de serem os certos
def process_eyes(eyes):
    if len(eyes)==2:
        return eyes
    elif len(eyes) > 2:
        eyes_index_to_vertical_dist = {}
        for i,eye1 in enumerate(eyes):
            for j,eye2 in enumerate(eyes):
                if i != j and (j,i) not in eyes_index_to_vertical_dist.keys():
                    eye1_center = get_eye_center(eye1)
                    eye2_center = get_eye_center(eye2)
                    eyes_index_to_vertical_dist[(i,j)] = abs(eye1_center[1]-eye2_center[1])
        smallest_dist = eyes_index_to_vertical_dist[(0,1)]
        smallest_dist_pair = (0,1)
        for k,v in eyes_index_to_vertical_dist.items():
            for k,v in eyes_index_to_vertical_dist.items():
                if v < smallest_dist:
                    smallest_dist = v
                    smallest_dist_pair = k
        eyes = np.array([eyes[idx] for idx in smallest_dist_pair])
        return eyes
    else:
        return np.empty((2,0))

In [339]:
x,y = np.zeros((2,0))
x,y

(array([], dtype=float64), array([], dtype=float64))

In [341]:
eyes_list = [process_eyes(eye) for eye in eyes_list]

In [343]:
for lefteye, righteye in eyes_list:
    if len(lefteye) > 0 and len(righteye) > 0:
        print(get_intraocular_distance_in_pixels(lefteye, righteye)/)

48.25453346577915
53.600373133029585
48.52319033204639
52.08646657242167
46.52418725781247
47.16990566028302
48.507731342539614
47.01063709417264
52.2733201547405
50.70009861923347
50.039984012787215
45.52471856036015
40.65095324835569
48.373546489791295
55.08175741568164
54.45181356024793
46.57252408878007
44.0
48.81085944746312
47.52367830881781
53.781967238099426
45.044422518220834
50.502475186865844
46.8187996428785
51.30789412946121
47.52367830881781
36.6742416417845
45.304525160297175
51.52184002925361
52.009614495783374
46.52418725781247
47.10095540432275
53.52102390649865
55.326304774492215
51.56064390598706
48.62612466565683
51.50242712727236
51.478150704935004
55.036351623268054
50.064957804835906
43.502873468312416
43.502873468312416
52.03844732503075
55.520266569965244
45.52471856036015
54.037024344425184
47.0
51.05389309347525
43.502873468312416
41.527099585692234
47.095647357266465
54.52063829413592
50.0
57.0
47.71268175233918
48.067660646218265
47.16990566028302
48.09365

In [377]:
import random
i = int(random.random()*len(faces_list))
aux = faces_list[i].copy()
color = [0,0,0]
for idx,(ex,ey,ew,eh) in enumerate(eyes_list[i]):
    cv2.rectangle(aux,(ex,ey),(ex+ew,ey+eh),tuple(color),2)
    color[idx%3] = 128
cv2.imshow(names_list[i], aux)
cv2.waitKey(0)
cv2.destroyAllWindows()

[array([[[249, 228, 226],
         [249, 228, 226],
         [249, 228, 226],
         ...,
         [ 79,  90, 118],
         [ 81,  94, 120],
         [ 73,  88, 112]],
 
        [[249, 228, 226],
         [249, 228, 226],
         [249, 228, 226],
         ...,
         [ 73,  83, 110],
         [ 81,  95, 120],
         [ 86, 101, 124]],
 
        [[249, 228, 226],
         [249, 228, 226],
         [249, 228, 226],
         ...,
         [ 70,  80, 105],
         [ 84,  97, 121],
         [ 91, 107, 130]],
 
        ...,
 
        [[162, 146, 159],
         [ 73,  63,  78],
         [102,  92, 107],
         ...,
         [ 10,  14,  33],
         [  6,  10,  29],
         [  6,  10,  28]],
 
        [[139, 123, 136],
         [ 81,  70,  86],
         [110, 100, 115],
         ...,
         [ 12,  15,  30],
         [  6,   9,  25],
         [ 13,  16,  32]],
 
        [[125, 109, 123],
         [100,  89, 104],
         [143, 132, 145],
         ...,
         [  7,   7,  23],
  